# Scraper script OpenMinVWS dossiers
Gebruikt Selenium om de dossier batch id te vinden en leidt naar de download pagina. BeautifulSoup download vervolgens het dossier.

Het script wacht met het downloaden van het volgend dossier totdat het vorige dossier is gedownload, in verband met de crawl delay en overbelasting.

In [31]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import requests
import gzip
import shutil
import xml.etree.ElementTree as ET
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import json
import zipfile
import os
import pandas as pd

Download van de gehele sitemap de sitemap van alle dossiers URLs op open.minvws.nl. Deze wordt opgeslagen in een lijst.

In [32]:
# URL van de sitemap van open.minvws.nl
sitemap_dossiers = "https://open.minvws.nl/sitemap.dossiers.xml.gz"

# Download het bestand
response = requests.get(sitemap_dossiers)

# Controleer of het bestand succesvol is gedownload
if response.status_code == 200:
    # Sla het bestand op als een .gz bestand
    with open("sitemap.dossiers.xml.gz", "wb") as file:
        file.write(response.content)
    
    # Pak het bestand uit
    with gzip.open("sitemap.dossiers.xml.gz", "rb") as f_in:
        with open("sitemap.dossiers.xml", "wb") as f_out:
            shutil.copyfileobj(f_in, f_out)
    
    # Parse de XML
    tree = ET.parse("sitemap.dossiers.xml")
    root = tree.getroot()

    # Defineer de namespace voor de XML
    namespaces = {'': 'http://www.sitemaps.org/schemas/sitemap/0.9'}

    # Zoek alle 'loc' tags en voeg ze toe aan een lijst
    dossier_urls = [url.find('loc', namespaces).text for url in root.findall('url', namespaces)]

    # Print de lijst van 'loc' URLs
#     print(dossier_urls)

else:
    print(f"Fout bij het downloaden: {response.status_code}")


Functie om een dossierpagina te openen, te klikken op de downloadknop en retourneert de batch-URL van dat dossier.

Hiervoor moet een ChromeDriver geinstalleerd zijn compatible met je Chrome browser versie. Eventueel makkelijk om te zetten naar een andere browser + driver.

In [33]:
# 🔹 **Selenium functie om batch-ID op te halen**
def find_batch_id(dossier_url):
    """
    Opent een dossierpagina, klikt op de downloadknop en retourneert de batch-URL.
    """
    chrome_driver_path = "/usr/local/bin/chromedriver"
    service = Service(chrome_driver_path)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")  # Zet op False als je de browser wilt zien
    driver = webdriver.Chrome(service=service, options=options)

    try:
        driver.get(dossier_url)
        time.sleep(3)  # Wacht zodat JavaScript kan laden

        download_button = driver.find_element(By.XPATH, "//button[@data-e2e-name='download-documents-button']")
        download_button.click()
        time.sleep(3)  # Wacht tot de nieuwe pagina met batch-ID laadt

        batch_url = driver.current_url
        return batch_url + "/download"

    except Exception as e:
        print(f"❌ Fout bij vinden van batch-ID: {e}")
        return None
    finally:
        driver.quit()

# Voorbeeldgebruik:
# dossier_url = "https://open.minvws.nl/dossier/VWS-WOO/1841132-219465-wjz"
# batch_url = find_batch_id(dossier_url)
# print(f"✅ Batch-URL: {batch_url}")


De besluitbrief kan gewoon met BeautifulSoup opgehaald worden

In [34]:
# 🔹 **Download link voor besluitbrief**
def get_document_download_link(document_page_url):
    """
    Haalt de directe downloadlink van de besluitbrief op.
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(document_page_url, headers=headers)
    if response.status_code != 200:
        print(f"❌ Kan de documentpagina niet bereiken: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    download_link_element = soup.find("a", {"data-e2e-name": "download-file-link"})
    return "https://open.minvws.nl" + download_link_element["href"] if download_link_element else None

Functie om metadata op de dossierpagina op te halen.

In [35]:
# 🔹 **Metadata en besluitbrief scraper**
def get_dossier_metadata_and_besluitbrief(dossier_url):
    """
    Scraped metadata en de link naar de besluitbrief.
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(dossier_url, headers=headers)
    if response.status_code != 200:
        print(f"❌ Kan de pagina niet bereiken: {response.status_code}")
        return None, None

    soup = BeautifulSoup(response.text, "html.parser")

    # Titel
    title_element = soup.find("h1", {"data-e2e-name": "dossier-metadata-title"})
    dossier_title = title_element.get_text(strip=True) if title_element else "Niet gevonden"
    if ":" in dossier_title:
        dossier_title = dossier_title.split(":", 1)[1].strip()

    # Publicatiedatum
    published_date_element = soup.find("dd", {"data-e2e-name": "dossier-metadata-published-date"})
    published_date = published_date_element.find("time")["datetime"] if published_date_element else "Niet gevonden"

    # Beslissingsdatum
    decision_date_element = soup.find("td", {"data-e2e-name": "dossier-metadata-decision-date"})
    decision_date = decision_date_element.find("time")["datetime"] if decision_date_element else "Niet gevonden"

    # Onderwerp
    subject_element = soup.find("th", string="Onderwerp")
    subject = None
    if subject_element:
        subject_td = subject_element.find_next("td")  # Zoek de volgende <td> eronder
        if subject_td:
            subject_links = subject_td.find_all("a", class_="woo-a")
            subject = ", ".join(link.get_text(strip=True) for link in subject_links) if subject_links else None

    # Verantwoordelijke instantie
    responsible_element = soup.find("td", {"data-e2e-name": "dossier-metadata-responsible"})
    responsible = responsible_element.get_text(strip=True) if responsible_element else None

    # Link naar het besluitbriefdocument
    document_link_element = soup.find("a", {"data-e2e-name": "main-document-detail-link"})
    document_page_url = "https://open.minvws.nl" + document_link_element["href"] if document_link_element else None

    # Voeg dossier_url toe aan metadata
    metadata = {
        "title": dossier_title,
        "published_date": published_date,
        "decision_date": decision_date,
        "subject": subject,
        "responsible": responsible,
        "dossier_url": dossier_url
    }

    return metadata, document_page_url

# Voorbeeldgebruik
# dossier_url = "https://open.minvws.nl/dossier/VWS-WOO/1841132-219465-wjz"
# metadata, document_page_url = get_dossier_metadata_and_besluitbrief(dossier_url)

# # Print metadata
# if metadata:
#     print("📂 Dossier Metadata:")
#     print(f"🔹 Titel: {metadata['title']}")
#     print(f"📅 Publicatiedatum: {metadata['published_date']}")
#     print(f"📜 Beslissingsdatum: {metadata['decision_date']}")
#     print(f"📚 Onderwerp: {metadata['subject']}")
#     print(f"🏢 Verantwoordelijke instantie: {metadata['responsible']}")
#     print(f"🔗 Dossier URL: {metadata['dossier_url']}")  # Print de dossier_url

# # Print link naar besluitbrief
# if document_page_url:
#     print(f"📑 Besluitbrief: {document_page_url}")


📂 Dossier Metadata:
🔹 Titel: Wob-deelbesluit aangaande Vaccinaties en medicatie over de periode februari 2020
📅 Publicatiedatum: 2024-09-12
📜 Beslissingsdatum: 2021-03-29
📚 Onderwerp: Vaccinaties en medicatie
🏢 Verantwoordelijke instantie: ministerie van Volksgezondheid, Welzijn en Sport
🔗 Dossier URL: https://open.minvws.nl/dossier/VWS-WOO/1841132-219465-wjz
📑 Besluitbrief: https://open.minvws.nl/dossier/VWS-WOO/1841132-219465-wjz/document


In [ ]:
def sanitize_filename(title, max_length=150):
    """
    Zorgt ervoor dat de bestandsnaam niet te lang wordt. Sommige dossiers hebben super lange namen.
    """
    title = title.replace("/", "-")  # Vermijd ongeldige tekens
    return title[:max_length].strip()  # Knip af tot de max lengte

Functie om met de batch-URL de dossier zip file te downloaden.

In [ ]:
# 🔹 **ZIP-bestand + besluitbrief downloaden**
def download_zip_and_document(dossier_url, save_path):
    """
    Zoek de ZIP-downloadlink met Selenium, download ZIP en besluitbrief en voeg metadata toe.
    """
    zip_url = find_batch_id(dossier_url)
    if not zip_url:
        print("❌ Geen batch-ID gevonden, ZIP wordt niet gedownload.")
        return

    metadata, document_page_url = get_dossier_metadata_and_document(dossier_url)
    if not metadata:
        print("⚠️ Geen metadata gevonden, ZIP wordt zonder metadata opgeslagen.")
        return

    zip_response = requests.get(zip_url, stream=True)
    if zip_response.status_code != 200:
        print(f"❌ Fout bij downloaden van ZIP: {zip_response.status_code}")
        return

    # Directe opslag als .zip bestand
    safe_title = sanitize_filename(metadata['title'])
    final_zip_path = os.path.join(save_path, safe_title + ".zip")

    with open(final_zip_path, "wb") as f:
        for chunk in zip_response.iter_content(1024):
            f.write(chunk)

    print(f"✅ ZIP gedownload: {final_zip_path}")

    metadata_json = json.dumps(metadata, indent=4)

    with zipfile.ZipFile(final_zip_path, 'a') as zipf:
        zipf.writestr("metadata.json", metadata_json)

    if document_page_url:
        print(f"📄 Bezoek besluitbrief-pagina: {document_page_url}")
        document_download_url = get_document_download_link(document_page_url)

        if document_download_url:
            doc_response = requests.get(document_download_url, stream=True)
            if doc_response.status_code == 200:
                temp_doc_path = final_zip_path + "_besluitbrief.pdf"
                with open(temp_doc_path, "wb") as f:
                    for chunk in doc_response.iter_content(1024):
                        f.write(chunk)

                print(f"✅ Besluitbrief gedownload: {temp_doc_path}")

                with zipfile.ZipFile(final_zip_path, 'a') as zipf:
                    zipf.write(temp_doc_path, arcname="besluitbrief.pdf")

                os.remove(temp_doc_path)
                print(f"📂 Besluitbrief toegevoegd aan ZIP: {final_zip_path}")

    print(f"✅ ZIP compleet: {final_zip_path}")
    
# Voorbeeldgebruik:
# dossier_url = "https://open.minvws.nl/dossier/VWS-WOO/1841132-219465-wjz"
# save_path = ""

# download_zip_and_document(dossier_url, save_path)

Run deze cell om alle dossiers in de URL lijst in het begin af te gaan en te downloaden. 

In totaal duurde dit ongeveer 8-10 uur.

In [ ]:
# Vul hier de map in waar de zip dossiers in moeten staan
output_dir = ""

start_dossier_nummer = 1

for dossier_url in dossier_urls:
    print(f"Dossiernummer: {start_dossier_nummer}")
    print(f"📂 Start met downloaden van dossier: {dossier_url}")
    download_zip_and_document(dossier_url, output_dir)
    print("✅ Download compleet. Start volgende...")
    start_dossier_nummer += 1
    
print("🎉 Alle dossiers zijn gedownload!")

Cell om dataframe van de dossiers + metadata te maken.

In [ ]:
def extract_metadata_from_zip(zip_path):
    """
    Opent een ZIP-bestand, leest de metadata.json en telt het aantal documenten (excl. metadata.json).
    """
    with zipfile.ZipFile(zip_path, 'r') as zip_file:
        file_list = zip_file.namelist()
        
        # Zoek metadata.json
        if "metadata.json" not in file_list:
            print(f"⚠️ Geen metadata.json in {zip_path}, overslaan...")
            return None
        
        with zip_file.open("metadata.json") as metadata_file:
            metadata = json.load(metadata_file)

        # Tel het aantal documenten in de ZIP, exclusief metadata.json
        document_count = sum(1 for file in file_list if file != "metadata.json")
    
    return metadata, document_count

def create_dossier_dataframe(output_dir, csv_path="df_dossiers.csv"):
    """
    Loopt door alle ZIP-bestanden in output_dir, haalt metadata op en slaat deze op in een CSV-bestand.
    """
    dossiers = []
    dossier_id = 1  # Start ID bij 1

    for filename in os.listdir(output_dir):
        if filename.endswith(".zip"):
            zip_path = os.path.join(output_dir, filename)
            metadata, document_count = extract_metadata_from_zip(zip_path)

            if metadata:
                dossiers.append({
                    "dossier_id": dossier_id,
                    "dossier_name": metadata.get("title", "Onbekend"),
                    "dossier_published_date": metadata.get("published_date", "Onbekend"),
                    "dossier_decision_date": metadata.get("decision_date", "Onbekend"),
                    "document_count": document_count,
                    "dossier_sourceURL": metadata.get("dossier_url", "Onbekend"),
                    "dossier_responsible": metadata.get("responsible", "Onbekend"),
                    "dossier_subject": metadata.get("subject", "Onbekend")
                })
                dossier_id += 1  # Verhoog ID

    # Zet data om in DataFrame en sla op als CSV
    df = pd.DataFrame(dossiers)
    df.to_csv(csv_path, index=False, encoding="utf-8")
    print(f"✅ Dossier DataFrame opgeslagen als {csv_path}")

# 🔹 **Gebruik**
output_dir = "/path/naar/je/output_dir"  # 🔹 Pas dit aan naar je eigen output map
create_dossier_dataframe(output_dir)
